# Métricas de Pré-Treino


In [43]:
!pip install numpy pandas scikit-learn > /dev/null 2>&1

1111.95s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [59]:
import numpy as np
import pandas as pd
np.random.seed(42)  # For reproducibility

df = pd.read_csv('datasets/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


# Class Imbalance

In [60]:
def class_imbalance(data: pd.DataFrame, attribute: str, privileged_class) -> float:
    class_counts = data[attribute].value_counts()
        
    n_a = class_counts[privileged_class]
    n_d = class_counts.sum() - n_a
    
    ci = (n_a - n_d) / (n_a + n_d)
    
    return ci

In [61]:
df_ci = pd.concat([df[df['sex'] == 1].sample(8), df[df['sex'] == 0].sample(2)], ignore_index=True)

class_imbalance(df_ci, "sex", 1)

np.float64(0.6)

# KL Divergence

In [62]:
def kl_divergence_from_df(data: pd.DataFrame, target, protected_attr, p, d) -> float:
    # número de instancias com sex == 1 e target==1 dividido por instancias com sex == 1
    p_p_y1 = (data[(data[protected_attr] == p) & (data[target] == 1)].shape[0]) / (data[data[protected_attr] == p].shape[0])
    p_d_y1 = (data[(data[protected_attr] == d) & (data[target] == 1)].shape[0]) / (data[data[protected_attr] == d].shape[0])
    p_p_y0 = (data[(data[protected_attr] == p) & (data[target] == 0)].shape[0]) / (data[data[protected_attr] == p].shape[0])
    p_d_y0 = (data[(data[protected_attr] == d) & (data[target] == 0)].shape[0]) / (data[data[protected_attr] == d].shape[0])
        
    # Calcular a divergência KL
    return p_p_y1 * np.log(p_p_y1 / p_d_y1) + p_p_y0 * np.log(p_p_y0 / p_d_y0)

In [63]:
df_only_male = df.loc[df['sex'] == 1]
df_only_female = df.loc[df['sex'] == 0]

df_mixed = pd.concat(
    [
        df_only_female.loc[df_only_female["target"] == 1].sample(7),
        df_only_female.loc[df_only_female["target"] == 0].sample(3),
        df_only_male.loc  [df_only_male  ["target"] == 1].sample(2),
        df_only_male.loc  [df_only_male  ["target"] == 0].sample(8),
    ]
)

kl_divergence_from_df(df_mixed, 'target', 'sex', 1, 0)

np.float64(0.5341108087103075)

# KS

In [64]:
def ks_test(data: pd.DataFrame, target: str, protected_attr: str, p: int, d: int) -> float:
    # Calcular a distribuição acumulada para cada classe
    p_dist = data[data[protected_attr] == p][target].value_counts(normalize=True).sort_index().cumsum()
    d_dist = data[data[protected_attr] == d][target].value_counts(normalize=True).sort_index().cumsum()
    
    # Calcular a diferença máxima entre as duas distribuições
    return np.max(np.abs(p_dist - d_dist))

In [65]:
ks_test(df_mixed, 'target', 'sex', 1, 0)

np.float64(0.5)

# CDDL

In [66]:
def _divide(a, b) -> float:
    if b == 0 and a == 0:
        return 0.0
    if b == 0:
        if a < 0:
            return float('-inf')
        return float('inf')
    return a / b
    

def CDDL(df: pd.DataFrame, target: str, protected_attribute, privileged_group, group_variable) -> float:
    unique_groups = np.unique(df[group_variable])
    CDD = np.array([])
    counts = np.array([])
    for subgroup_variable in unique_groups:
        counts = np.append(
            counts, (df[group_variable].values == subgroup_variable).sum())
        numA = len(df[(df[target] == 0) & (
            df[protected_attribute] != privileged_group) & (df[group_variable] == subgroup_variable)])
        denomA = len(df[(df[target] == 0) &
                        (df[group_variable] == subgroup_variable)])
        A = numA / denomA if denomA != 0 else 0
        numD = len(df[(df[target] != 0) & (
            df[protected_attribute] != privileged_group) & (df[group_variable] == subgroup_variable)])
        denomD = len(df[(df[target] != 0) &
                        (df[group_variable] == subgroup_variable)])
        D = numD / denomD if denomD != 0 else 0
        CDD = np.append(CDD, D - A)
    return _divide(np.sum(counts * CDD), np.sum(counts))

In [67]:
df_cddl = pd.DataFrame({
    "target": [1]*4 + [0]*1 + [1]*2 + [0]*3 + [1]*3 + [0]*2 + [1]*1 + [0]*4,
    "sex": ["F"]*10 + ["M"]*10,
    "age>20": [1]*5 + [0]*5 + [1]*5 + [0]*5
})

cddl_value = CDDL(
    df=df_cddl,
    target="target",
    protected_attribute="sex",
    privileged_group="M",
    group_variable="age>20"
)
print(f"CDDL: {cddl_value:.2f}")


CDDL: 0.24


# Post-Training

## DPPL

In [68]:
def DPPL(df: pd.DataFrame, target: str, protected_attr:str, privileged_group: str) -> float:
    q_p = len(df[(df[target] == 1) & (df[protected_attr] == privileged_group)]) / len(df[df[protected_attr] == privileged_group])
    q_d = len(df[(df[target] == 1) & (df[protected_attr] != privileged_group)]) / len(df[df[protected_attr] != privileged_group])

    return q_p - q_d

## DI

In [69]:
def DI(df: pd.DataFrame, target: str, protected_attr:str, privileged_group: str) -> float:
    q_p = len(df[(df[target] == 1) & (df[protected_attr] == privileged_group)]) / len(df[df[protected_attr] == privileged_group])
    q_d = len(df[(df[target] == 1) & (df[protected_attr] != privileged_group)]) / len(df[df[protected_attr] != privileged_group])

    return q_d / q_p

In [70]:
men_data = pd.DataFrame({
    'sex': [1] * 100,
    'target': [1] * 70 + [0] * 30  # 70% negative outcomes, 30% positive outcomes
})

women_data = pd.DataFrame({
    'sex': [0] * 100,
    'target': [1] * 50 + [0] * 50  # 50% negative outcomes, 50% positive outcomes
})

df_dppl = pd.concat([men_data, women_data], ignore_index=True)

dppl = DPPL(df_dppl, 'target', 'sex', 1) 
di = DI(df_dppl, 'target', 'sex', 1) 

print(f"DPPL: {dppl:.2f}, DI: {di:.2f}")


DPPL: 0.20, DI: 0.71


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
model = LogisticRegression(max_iter=10000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

male_indices = X_test['sex'] == 1
female_indices = X_test['sex'] == 0

cm_male = confusion_matrix(y_test[male_indices], y_pred[male_indices], labels=[1, 0])
cm_female = confusion_matrix(y_test[female_indices], y_pred[female_indices], labels=[1, 0])

# print("Confusion Matrix for Male:")
# print(cm_male)
# print("\nConfusion Matrix for Female:")
# print(cm_female)

# true positives for male
TPp = cm_male[0][0]
FNp = cm_male[0][1]
FPp = cm_male[1][0]
TNp = cm_male[1][1]
recall_male = (TPp / (TPp + FNp))
# print("TPp: ", TPp)
# print("FNp:", FNp)
# print("FPp:", FPp)
# print("TNp:", TNp)

# true positives for female
TPd = cm_female[0][0]
FNd = cm_female[0][1]
FPd = cm_female[1][0]
TNd = cm_female[1][1]

# print("TPd: ", TPd)
# print("FNd:", FNd)
# print("FPd:", FPd)
# print("TNd:", TNd)
recall_female = (TPd / (TPd + FNd))

# print("Recall for Female: ", recall_female)
# print("Recall for Male: ", recall_male)
RD = recall_male - recall_female
print("Recall Diference: ", RD)

DAR = (TPp / (TPp+FPp)) - (TPd / (TPd+FPd))
print("DAR: ", DAR)

DRR = (TNd / (TNd + FNd)) - (TNp / (TNp + FNp))
print("DRR: ", DRR)

Recall Diference:  -0.1955211024978466
DAR:  -0.2142857142857143
DRR:  0.09239130434782605
